In [26]:
import pandas as pd
import numpy as np
import os

In [27]:
os.getcwd()

'C:\\Users\\User\\OneDrive\\Desktop\\CS50AI\\degrees'

# Dataset Check

In [216]:
df_movies = pd.read_csv('.\large\movies.csv')
df_people = pd.read_csv(".\large\people.csv")
df_stars = pd.read_csv(".\large\stars.csv")

In [267]:
df_movies.head()

,id,title,year
0,15724,Dama de noche,1993
1,23331,Pesn o geroyakh,1983
2,31458,El huésped del sevillano,1970
3,35423,Kate & Leopold,2001
4,36606,"Another Time, Another Place",1983


In [266]:
df_people.head()

,id,name,birth
0,1,Fred Astaire,1899.0
1,2,Lauren Bacall,1924.0
2,3,Brigitte Bardot,1934.0
3,4,John Belushi,1949.0
4,5,Ingmar Bergman,1918.0


In [268]:
df_stars.head()

,person_id,movie_id
0,844752,15724
1,869732,15724
2,194720,15724
3,650495,15724
4,8738,31458


# Function Check

In [241]:
names  = {}
people = {}
movies = {}

# specify directory small/large
directory = ".\large"


In [242]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}\people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}\movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}\stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

load_data(directory)

In [243]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [244]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

In [245]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node


In [270]:
source = person_id_for_name(input("Name: "))
if source is None:
    print("Person not found.")
target = person_id_for_name(input("Name: "))
if target is None:
    print("Person not found.")

Name: Kevin Bacon
Which 'Kevin Bacon'?
ID: 9323132, Name: Kevin Bacon, Birth: 
ID: 102, Name: Kevin Bacon, Birth: 1958
Intended Person ID: 102
Name: Tom Hanks


In [ ]:
num_explored = 0
start = Node(state = source, parent = None, action = None)
frontier = QueueFrontier()
frontier.add(start)

explored = set()
while True:
    if frontier.empty():
        print("There is no solution")
        
    node = frontier.remove()
    num_explored += 1
    
    if node.state == target:
        sol_list = []
        while node.parent is not None:
            sol_list.append((node.action,node.state))
            node = node.parent
        sol_list.reverse()
        print(sol_list)

    explored.add(node.state)
    
    for action, state in neighbors_for_person(node.state):
        if not frontier.contains_state(state) and state not in explored:
            child = Node(state=state, parent=node, action=action)
            frontier.add(child)

[('112384', '158')]


In [258]:
%run C:\Users\User\OneDrive\Desktop\CS50AI\degrees\degrees.py

Loading data...
Data loaded.
Name: Tom Hanks
Name: Valeria Golino
2 degrees of separation.
1: Tom Hanks and David Morse starred in The Green Mile
2: David Morse and Valeria Golino starred in The Indian Runner


In [269]:
%run C:\Users\User\OneDrive\Desktop\CS50AI\degrees\degrees.py

Loading data...
Data loaded.
Name: Tom Cruise
Name: Chris Sarandon
2 degrees of separation.
1: Tom Cruise and Tim Curry starred in Legend
2: Tim Curry and Chris Sarandon starred in The Chosen One


# Another Solution

In [263]:
def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    If no possible path, returns None.
    """
 
    explored = set([])
    frontier = [source]
    parents = {}
    while len(frontier) > 0:
        person = frontier.pop(0)
        if person == target:
            break
        explored.add(person)
        for (m, p) in neighbors_for_person(person):
            if not p in frontier and not p in explored:
                frontier.append(p)
                parents[p] = (m, person)
    if not target in parents:
        return None
    path = []
    person = target
    while person != source:
        m, p = parents[person]
        path.append((m, person))
        person = p
    path = path[::-1]
    return path

In [264]:
shortest_path('102','420')

[('104257', '129'), ('95953', '420')]